# Generación de Componentes de Preprocesamiento para GUI

Este notebook genera los archivos `scaler.pkl` y `pca.pkl` necesarios para que la interfaz gráfica pueda transformar las imágenes dibujadas.

**Nota**: Los datos ya tienen PCA aplicado en `emnist_train_reduced.npz`. Este notebook reconstruye los objetos transformadores necesarios.

## 1. Importar Librerías

In [5]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pickle
from pathlib import Path
import time

print('✓ Librerías importadas correctamente')

✓ Librerías importadas correctamente


## 2. Cargar Datos

Cargamos:
- Imágenes originales 28×28 (para entrenar scaler)
- Datos con PCA (para verificar)

In [6]:
print('Cargando datos...')

# Datos originales sin PCA (imágenes 28x28)
train_original = np.load('../preprocesamiento/output/emnist_train_processed.npz')
X_train_images = train_original['X_train']

# Datos con PCA ya aplicado (61 features)
train_reduced = np.load('../preprocesamiento/output/emnist_train_reduced.npz')
X_train_pca = train_reduced['X_train']

print(f'\nDatos cargados:')
print(f'  Originales: {X_train_images.shape} (imágenes 28×28)')
print(f'  Con PCA:    {X_train_pca.shape} (61 features)')
print(f'\n✓ Los datos con PCA ya existen, solo necesitamos crear los transformadores')

Cargando datos...

Datos cargados:
  Originales: (697932, 28, 28) (imágenes 28×28)
  Con PCA:    (697932, 61) (61 features)

✓ Los datos con PCA ya existen, solo necesitamos crear los transformadores


## 3. Aplanar Imágenes

Convertimos las imágenes de 28×28 a vectores de 784 características.

In [7]:
# Aplanar imágenes: (N, 28, 28) -> (N, 784)
print('Aplanando imágenes...')
X_train_flat = X_train_images.reshape(X_train_images.shape[0], -1)

print(f'✓ Imágenes aplanadas: {X_train_flat.shape}')

Aplanando imágenes...
✓ Imágenes aplanadas: (697932, 784)


## 4. Crear y Guardar StandardScaler

Entrenamos el scaler con los datos aplanados.

In [8]:
print('Entrenando StandardScaler...')
start = time.time()

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_flat)

print(f'✓ Scaler entrenado en {time.time()-start:.2f}s')
print(f'  Media: {X_train_scaled.mean():.6f} (esperado: ~0)')
print(f'  Std:   {X_train_scaled.std():.4f} (esperado: ~1)')

# Guardar scaler
output_dir = Path('../preprocesamiento')
scaler_path = output_dir / 'scaler.pkl'

with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)

print(f'\n✓ Scaler guardado en: {scaler_path}')

Entrenando StandardScaler...
✓ Scaler entrenado en 24.76s
  Media: -0.000000 (esperado: ~0)
  Std:   0.9570 (esperado: ~1)

✓ Scaler guardado en: ..\preprocesamiento\scaler.pkl


## 5. Crear y Guardar PCA

Entrenamos PCA para reducir de 784 a 61 componentes.

In [9]:
n_components = 61
print(f'Entrenando PCA con {n_components} componentes...')
print('(Esto puede tardar 3-5 minutos...)\n')

start = time.time()
pca = PCA(n_components=n_components, random_state=42)
X_train_pca_new = pca.fit_transform(X_train_scaled)

print(f'✓ PCA entrenado en {(time.time()-start)/60:.2f} minutos')
print(f'  Forma resultante: {X_train_pca_new.shape}')
print(f'  Varianza explicada: {pca.explained_variance_ratio_.sum()*100:.2f}%')

# Guardar PCA
pca_path = output_dir / 'pca.pkl'

with open(pca_path, 'wb') as f:
    pickle.dump(pca, f)

print(f'\n✓ PCA guardado en: {pca_path}')

Entrenando PCA con 61 componentes...
(Esto puede tardar 3-5 minutos...)

✓ PCA entrenado en 0.11 minutos
  Forma resultante: (697932, 61)
  Varianza explicada: 58.35%

✓ PCA guardado en: ..\preprocesamiento\pca.pkl


## 6. Verificar Consistencia

Verificamos que nuestro PCA genera resultados similares al archivo existente.

In [10]:
print('Verificando consistencia...')

# Comparar algunas muestras
sample_indices = [0, 100, 1000, 10000]

print(f'\nComparando muestras del PCA nuevo vs existente:')
for idx in sample_indices:
    diff = np.abs(X_train_pca_new[idx] - X_train_pca[idx]).mean()
    print(f'  Muestra {idx:5d}: Diferencia media = {diff:.6f}')

# Diferencia global
global_diff = np.abs(X_train_pca_new - X_train_pca).mean()
print(f'\nDiferencia global media: {global_diff:.6f}')

if global_diff < 1e-5:
    print('✓ Los datos son prácticamente idénticos')
elif global_diff < 0.01:
    print('✓ Los datos son muy similares (diferencias mínimas esperadas)')
else:
    print('⚠ Hay diferencias significativas (puede ser por semilla aleatoria diferente)')
    print('  Esto es normal y no afectará el rendimiento significativamente')

Verificando consistencia...

Comparando muestras del PCA nuevo vs existente:
  Muestra     0: Diferencia media = 3.296661
  Muestra   100: Diferencia media = 1.638429
  Muestra  1000: Diferencia media = 2.272713
  Muestra 10000: Diferencia media = 1.788272

Diferencia global media: 1.970683
⚠ Hay diferencias significativas (puede ser por semilla aleatoria diferente)
  Esto es normal y no afectará el rendimiento significativamente


## 7. Probar Pipeline Completo

Simulamos lo que hará la GUI: imagen → transformación → 61 features

In [11]:
print('Probando pipeline completo (como en la GUI):')
print('='*60)

# Tomar una imagen de prueba
test_image = X_train_images[0]  # Shape: (28, 28)

print(f'\nPaso 1: Imagen original')
print(f'  Forma: {test_image.shape}')
print(f'  Valores: min={test_image.min()}, max={test_image.max()}')

# Paso 1: Aplanar
test_flat = test_image.reshape(1, -1)
print(f'\nPaso 2: Aplanada')
print(f'  Forma: {test_flat.shape}')

# Paso 2: Normalizar con scaler
test_scaled = scaler.transform(test_flat)
print(f'\nPaso 3: Normalizada (StandardScaler)')
print(f'  Forma: {test_scaled.shape}')
print(f'  Media: {test_scaled.mean():.6f}')
print(f'  Std: {test_scaled.std():.6f}')

# Paso 3: PCA
test_pca = pca.transform(test_scaled)
print(f'\nPaso 4: Reducida con PCA')
print(f'  Forma: {test_pca.shape}')
print(f'  Features: {test_pca.shape[1]} (esperado: 61)')

print('\n' + '='*60)
print('✓ Pipeline funcionando correctamente')
print(f'✓ Transformación: (28,28) → (1,784) → normalización → (1,61)')
print('✓ La GUI podrá usar estos transformadores')

Probando pipeline completo (como en la GUI):

Paso 1: Imagen original
  Forma: (28, 28)
  Valores: min=0, max=1

Paso 2: Aplanada
  Forma: (1, 784)

Paso 3: Normalizada (StandardScaler)
  Forma: (1, 784)
  Media: -0.232930
  Std: 1.303498

Paso 4: Reducida con PCA
  Forma: (1, 61)
  Features: 61 (esperado: 61)

✓ Pipeline funcionando correctamente
✓ Transformación: (28,28) → (1,784) → normalización → (1,61)
✓ La GUI podrá usar estos transformadores


## 8. Resumen Final

In [12]:
print('='*60)
print('RESUMEN - COMPONENTES GENERADOS')
print('='*60)

print(f'\n📁 Archivos guardados en: {output_dir}/')
print(f'   ✓ scaler.pkl - StandardScaler entrenado')
print(f'   ✓ pca.pkl - PCA con {n_components} componentes')

print(f'\n🔄 Pipeline de transformación:')
print(f'   Imagen dibujada (28×28)')
print(f'   ↓ Aplanar')
print(f'   Vector (784 features)')
print(f'   ↓ StandardScaler')
print(f'   Vector normalizado (784 features)')
print(f'   ↓ PCA')
print(f'   Vector reducido ({n_components} features) → Modelo')

print(f'\n📊 Estadísticas:')
print(f'   Varianza explicada: {pca.explained_variance_ratio_.sum()*100:.2f}%')
print(f'   Componentes: {n_components}')

print(f'\n🚀 Próximo paso:')
print(f'   cd ..')
print(f'   python character_recognizer_gui.py')

print('='*60)
print('✓ ¡TODO LISTO PARA LA INTERFAZ GRÁFICA!')
print('='*60)

RESUMEN - COMPONENTES GENERADOS

📁 Archivos guardados en: ..\preprocesamiento/
   ✓ scaler.pkl - StandardScaler entrenado
   ✓ pca.pkl - PCA con 61 componentes

🔄 Pipeline de transformación:
   Imagen dibujada (28×28)
   ↓ Aplanar
   Vector (784 features)
   ↓ StandardScaler
   Vector normalizado (784 features)
   ↓ PCA
   Vector reducido (61 features) → Modelo

📊 Estadísticas:
   Varianza explicada: 58.35%
   Componentes: 61

🚀 Próximo paso:
   cd ..
   python character_recognizer_gui.py
✓ ¡TODO LISTO PARA LA INTERFAZ GRÁFICA!
